# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [231]:
import time
import pandas as pd
import fake_useragent
import lxml
from bs4 import BeautifulSoup
import requests
import re # need for regular expressions (text pattern searching)

In [232]:
html_page = requests.get('http://books.toscrape.com/') # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'lxml') # Pass the page contents to beautiful soup for parsing

In [233]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="

In [234]:
## pprint is a helpful package for printing complex data
from IPython import display as dp
!pip install -U pprint
dp.clear_output()

from pprint import pprint
    
print(soup.prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

In [235]:
# select an encapsulating container
# the warning section above the books with the warning has all the books and their info nested 
warning = soup.find('div', class_="alert alert-warning")
warning # Previewing is optional but can help you verify you are selecting what you think you are

<div class="alert alert-warning" role="alert"><strong>Warning!</strong> This is a demo website for web scraping purposes. Prices and ratings here were randomly assigned and have no real meaning.</div>

In [236]:
warning.nextSibling # the immediate next sibling to the warning is just a blank line

'\n'

In [237]:
warning.nextSibling.nextSibling # the next next sibling to the warning is what we're after

<div>
<ol class="row">
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_containe

In [238]:
# now we have our container that holds the book info we're interested in
book_container = warning.nextSibling.nextSibling

# within this container it seems like we can get the book titles out of h3 elements from title attr
book_container.find('h3')

<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [239]:
titles = book_container.find_all('h3')
titles[0] # preview first entry again

<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [240]:
# the title is in an attr of a link, so select the link
titles[0].find('a')

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [241]:
# slice out the title attr from the link and we'll have just the title of the first book
titles[0].find('a').attrs['title']

'A Light in the Attic'

In [242]:
# put together a cleaned up list of titles with a list comprehension
final_titles = [h3.find('a').attrs['title'] for h3 in book_container.find_all('h3')]
print(len(final_titles), final_titles[:5]) # got titles for all 20 books on the page

20 ['A Light in the Attic', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History of Humankind']


## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [243]:
def retrieve_titles(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    final_titles = [h3.find('a').attrs['title'] for h3 in book_container.find_all('h3')]
    return final_titles

In [244]:
retrieve_titles(soup)

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [245]:
book_container

<div>
<ol class="row">
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_containe

In [246]:
# ratings are inside paragraph tags, specifically within the class attr
book_container.find('p')

<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>

In [247]:
# define a pattern to search for to pick out the star rating
regex = re.compile("star-rating (.*)")
book_container.find_all('p', {"class" : regex}) # Initial Trial in developing the script


[<p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating Four">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating Five">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class

In [248]:
ratings = book_container.find_all('p', {'class': regex})
first_rating = ratings[0]
first_rating.attrs['class'] # so to get the star rating, access the last element of the attr of p tags with our specified class

['star-rating', 'Three']

In [249]:
star_ratings = []

for p in book_container.find_all('p', {'class': regex}):
    star_ratings.append(p.attrs['class'][-1])
star_ratings

['Three',
 'One',
 'One',
 'Four',
 'Five',
 'One',
 'Four',
 'Three',
 'Four',
 'One',
 'Two',
 'Four',
 'Five',
 'Five',
 'Five',
 'Three',
 'One',
 'One',
 'Two',
 'Two']

In [250]:
# But the star ratings are strings, need to convert to more useful integers
## Manually create a dictionary to translate to numeric
star_dict = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
## List comprehension to swap the key from the dict with its corresponding value
#### return the value of the key for each key provided by each element of the list
star_ratings = [star_dict[s] for s in star_ratings]
star_ratings

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

In [251]:
def retrieve_ratings(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    
    regex = re.compile("star-rating (.*)")
    star_ratings = []
    for p in book_container.find_all('p', {'class': regex}):
        star_ratings.append(p.attrs['class'][-1])
    
    star_dict = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    star_ratings = [star_dict[s] for s in star_ratings]
    return star_ratings

In [252]:
retrieve_ratings(soup)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [253]:
book_container

<div>
<ol class="row">
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_containe

In [254]:
# prices are inside p tags with class="price_color"
book_container.find_all('p')

[<p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="price_color">£51.77</p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="price_color">£53.74</p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="star-rating One">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>, <p class="price_color">£50.10</p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="star-rating Four">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="ico

In [255]:
# define a pattern to search for to pick out the star rating
regex = re.compile("price_color")
book_container.find_all('p', {"class" : regex}) # Initial Trial in developing the script

[<p class="price_color">£51.77</p>,
 <p class="price_color">£53.74</p>,
 <p class="price_color">£50.10</p>,
 <p class="price_color">£47.82</p>,
 <p class="price_color">£54.23</p>,
 <p class="price_color">£22.65</p>,
 <p class="price_color">£33.34</p>,
 <p class="price_color">£17.93</p>,
 <p class="price_color">£22.60</p>,
 <p class="price_color">£52.15</p>,
 <p class="price_color">£13.99</p>,
 <p class="price_color">£20.66</p>,
 <p class="price_color">£17.46</p>,
 <p class="price_color">£52.29</p>,
 <p class="price_color">£35.02</p>,
 <p class="price_color">£57.25</p>,
 <p class="price_color">£23.88</p>,
 <p class="price_color">£37.59</p>,
 <p class="price_color">£51.33</p>,
 <p class="price_color">£45.17</p>]

In [256]:
regex = re.compile("price_color")
cl_price = book_container.find_all('p', {"class" : regex})

In [257]:
# the actual price is found within the text inside the element, not inside the tag itself this time,
## so access by accessing the text
cl_price[0].text

'£51.77'

In [258]:
# get rid of the pound sign in coerce to be a float
float(cl_price[0].text[1:])

51.77

In [259]:
prices = []

for p in book_container.find_all('p', {'class': regex}):
    prices.append(float(p.text[1:]))
print(len(prices), prices[:5]) # got all 20 prices as floats

20 [51.77, 53.74, 50.1, 47.82, 54.23]


In [260]:
def retrieve_prices(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    
    regex = re.compile("price_color")
    prices = []
    for p in book_container.find_all('p', {'class': regex}):
        prices.append(float(p.text[1:]))
    
    return prices

In [261]:
retrieve_prices(soup)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [262]:
book_container

<div>
<ol class="row">
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>
<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_containe

In [263]:
# availability (In stock) is found within p elements with class="instock availability"
book_container.find_all('p', class_="instock availability")

[<p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>, <p class="instock availability">
 <i class="icon-

In [264]:
avails = book_container.find_all('p', class_="instock availability")
avails[0]

<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>

In [265]:
first_avail = avails[0]
avails[0].text

'\n\n    \n        In stock\n    \n'

In [266]:
# get rid of the extra whitespace with strip method
avails[0].text.strip()

'In stock'

In [267]:
avails = [a.text.strip() for a in book_container.find_all('p', class_="instock availability")]
print(len(avails), avails[0:5]) # got availability for all 20 books on the page

20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']


In [268]:
def retrieve_availabilities(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    
    avails = [a.text.strip() for a in book_container.find_all('p', class_="instock availability")]
    return avails
    

In [269]:
retrieve_availabilities(soup)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [278]:
# function using extend method
def url_parse(url, titles=[], star_ratings=[], prices=[], avails=[]):

#     for i in range(1,51):
    for i in range(1,3):
        if i == 1:
            url = url 
        else:
            url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
        html_page = requests.get('{}'.format(url), time.sleep(3))
        soup = BeautifulSoup(html_page.content, 'lxml')
        new_titles = retrieve_titles(soup)
        titles.extend(new_titles)
        new_star_ratings = retrieve_ratings(soup)
        star_ratings.extend(new_star_ratings)
        new_prices = retrieve_prices(soup)
        prices.extend(new_prices)
        new_avails = retrieve_availabilities(soup)
        avails.extend(new_avails)
    
    df = pd.DataFrame([titles, star_ratings, prices, avails]).transpose()
    df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
    return df

In [279]:
df = url_parse('http://books.toscrape.com/')

In [280]:
display(df.shape, df.head(10), df.tail(10))

(1000, 4)

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


,Title,Star_Rating,Price_(pounds),Availability
990,Bridget Jones's Diary (Bridget Jones #1),1,29.82,In stock
991,Bounty (Colorado Mountain #7),4,37.26,In stock
992,Blood Defense (Samantha Brinkman #1),3,20.3,In stock
993,"Bleach, Vol. 1: Strawberry and the Soul Reaper...",5,34.65,In stock
994,Beyond Good and Evil,1,43.38,In stock
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock
999,"1,000 Places to See Before You Die",5,26.08,In stock


In [283]:
# function using += instead of extend, fewer lines of code
def url_parse(url, titles=[], star_ratings=[], prices=[], avails=[]):

#     for i in range(1,51):
    for i in range(1,3):
        if i == 1:
            url = url 
        else:
            url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
        html_page = requests.get('{}'.format(url), time.sleep(3))
        soup = BeautifulSoup(html_page.content, 'lxml')
        titles += retrieve_titles(soup)
        star_ratings += retrieve_ratings(soup)
        prices += retrieve_prices(soup)
        avails += retrieve_availabilities(soup)
    
    df = pd.DataFrame([titles, star_ratings, prices, avails]).transpose()
    df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
    return df

In [284]:
df = url_parse('http://books.toscrape.com/')

In [285]:
display(df.shape, df.head(10), df.tail(10))

(40, 4)

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


,Title,Star_Rating,Price_(pounds),Availability
30,The Four Agreements: A Practical Guide to Pers...,5,17.66,In stock
31,The Five Love Languages: How to Express Heartf...,3,31.05,In stock
32,The Elephant Tree,5,23.82,In stock
33,The Bear and the Piano,1,36.89,In stock
34,Sophie's World,5,15.94,In stock
35,Penny Maybe,3,33.29,In stock
36,Maude (1883-1993):She Grew Up with the country,2,18.02,In stock
37,"In a Dark, Dark Wood",1,19.63,In stock
38,Behind Closed Doors,4,52.22,In stock
39,You can't bury them all: Poems,2,33.63,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [223]:
button_container = soup.find('ul', class_='pager')
button_container

<ul class="pager">
<li class="previous"><a href="page-49.html">previous</a></li>
<li class="current">
            
                Page 50 of 50
            
            </li>
</ul>

In [229]:
button_container.parent

<div>
<ul class="pager">
<li class="previous"><a href="page-49.html">previous</a></li>
<li class="current">
            
                Page 50 of 50
            
            </li>
</ul>
</div>

'\n'

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!